## Parte 1: Preparación de Datos y Definición del Área de Estudio

### Introducción
Este proyecto implementa metodologías de detección de cambios usando imágenes Sentinel-1 SAR para identificar cambios potenciales asociados a actividad agrícola en municipios seleccionados de los departamentos de Casanare y Meta, Colombia [1]. La detección de cambios mediante radar de apertura sintética (SAR) ofrece ventajas significativas sobre sensores ópticos, particularmente en regiones tropicales con cobertura nubosa frecuente [2].

### Área de estudio
Los municipios seleccionados corresponden a zonas de importancia agrícola en la región de la Orinoquía colombiana:

**Departamento del Meta:**
- Puerto López
- Castilla La Nueva
- San Carlos de Guaroa
- Cabuyaro

**Departamento de Casanare:**
- Tauramena
- Yopal
- Aguazul
- Nunchía
- Villanueva

### Metodología
Este notebook implementa la primera fase del procesamiento siguiendo la metodología propuesta por Canty et al. [3] para análisis estadístico de cambios en series temporales de Sentinel-1 en Google Earth Engine. Los datos SAR de polarización dual (VV y VH) serán utilizados para detectar cambios estadísticamente significativos en la superficie terrestre.

## 1. Carga de librerías

Trabaja dentro de un entorno virtual dedicado para mantener controladas las dependencias del proyecto. Activa el entorno y asegúrate de que las bibliotecas de análisis geoespacial, visualización y la API de Google Earth Engine estén instaladas antes de ejecutar las celdas. Estas herramientas permiten replicar los ejercicios de cambio multitemporal descritos en el tutorial oficial [4]. Importa los módulos siguiendo el orden estándar → científicos → geoespaciales → visualización para que cualquiera identifique las dependencias de un vistazo.

In [ ]:
# Instalación de dependencias necesarias
# !pip install earthengine-api geemap geopandas fiona

import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")

## 2. Inicialización de Google Earth Engine

Earth Engine es el motor que nos permite descargar y procesar las imágenes Sentinel-1. Usa el bloque `try/except` que llama primero a `ee.Initialize()` y, si falla, ejecuta `ee.Authenticate()` seguido de `ee.Initialize()`. Este patrón evita interrupciones cuando la sesión expira y es el que recomienda el tutorial de cambios con Sentinel-1 [4].

In [ ]:
# Inicializar Earth Engine
try:
    ee.Initialize()
    print("Earth Engine inicializado correctamente")
except:
    print("Autenticando Earth Engine...")
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine autenticado e inicializado")

## 3. Carga de la capa de municipios

Abrimos el GeoPackage oficial del DANE 2023 (`MGN_MPIO_POLITICO`) para trabajar con límites municipales recientes. Este archivo contiene los nueve municipios seleccionados en Casanare y Meta, que representan la diversidad agrícola de los Llanos Orientales según los reportes estadísticos del sector [5].

In [ ]:
# Ruta al archivo GeoPackage con los límites municipales
gpkg_path = "/home/famartinezal/Dropbox/Base/DANE_BASE_2023.gpkg"
layer_name = "MGN_MPIO_POLITICO"

# Cargar la capa completa
municipios_colombia = gpd.read_file(gpkg_path, layer=layer_name)

print(f"Total de municipios en Colombia: {len(municipios_colombia)}")
print(f"Sistema de referencia: {municipios_colombia.crs}")
print(f"\nColumnas disponibles: {list(municipios_colombia.columns)}")

## 4. Selección de municipios de interés

Filtra la capa con los códigos DANE de cada municipio y guarda la lista en una variable clara (`municipios_objetivo`). Esta lista sirve para repetir el análisis cuando se actualicen los datos y facilita que otros revisen la configuración sin buscar manualmente los filtros.

In [ ]:
# Definir los municipios objetivo
municipios_meta = [
    ('META', 'PUERTO LÓPEZ', '50', '573'),
    ('META', 'CASTILLA LA NUEVA', '50', '150'),
    ('META', 'SAN CARLOS DE GUAROA', '50', '680'),
    ('META', 'CABUYARO', '50', '124')
]

municipios_casanare = [
    ('CASANARE', 'TAURAMENA', '85', '410'),
    ('CASANARE', 'YOPAL', '85', '001'),
    ('CASANARE', 'AGUAZUL', '85', '010'),
    ('CASANARE', 'NUNCHÍA', '85', '225'),
    ('CASANARE', 'VILLANUEVA', '85', '440')
]

# Combinar todas las listas de municipios
todos_municipios = municipios_meta + municipios_casanare

# Crear DataFrame con la información
municipios_info = pd.DataFrame(
    todos_municipios, 
    columns=['departamento', 'municipio', 'cod_dpto', 'cod_mpio']
)

print("Municipios seleccionados:")
print(municipios_info.to_string(index=False))

In [ ]:
# Filtrar los municipios de interés
municipios_seleccionados = municipios_colombia[
    (municipios_colombia['dpto_cnmbr'].isin(['META', 'CASANARE'])) &
    (municipios_colombia['mpio_cnmbr'].isin([
        'PUERTO LÓPEZ', 'CASTILLA LA NUEVA', 'SAN CARLOS DE GUAROA', 'CABUYARO',
        'TAURAMENA', 'YOPAL', 'AGUAZUL', 'NUNCHÍA', 'VILLANUEVA'
    ]))
].copy()

print(f"\nMunicipios encontrados: {len(municipios_seleccionados)}")
print("\nDetalle de municipios:")
print(municipios_seleccionados[['dpto_cnmbr', 'mpio_cnmbr', 'dpto_ccdgo', 'mpio_ccdgo', 'mpio_narea']])

## 5. Reproyección y preparación de geometrías

Convierte el GeoDataFrame a WGS84 (EPSG:4326) antes de crear objetos de Earth Engine. Esta proyección es el punto de encuentro para la mayoría de los catálogos satelitales y garantiza que las geometrías coincidan con las imágenes Sentinel-1 descritas en el manual de la misión [6].

In [ ]:
# Reproyectar a WGS84 (EPSG:4326) para compatibilidad con Earth Engine
municipios_seleccionados = municipios_seleccionados.to_crs(epsg=4326)

# Calcular el área total y el centroide
area_total_km2 = municipios_seleccionados.geometry.area.sum() * 111.32 * 111.32  # Aproximación
centroide = municipios_seleccionados.geometry.unary_union.centroid

print(f"\nÁrea total aproximada: {area_total_km2:,.2f} km²")
print(f"Centroide del área de estudio: Lat {centroide.y:.4f}, Lon {centroide.x:.4f}")

# Calcular el bounding box
bounds = municipios_seleccionados.total_bounds
print(f"\nBounding Box:")
print(f"  Oeste: {bounds[0]:.4f}°")
print(f"  Sur: {bounds[1]:.4f}°")
print(f"  Este: {bounds[2]:.4f}°")
print(f"  Norte: {bounds[3]:.4f}°")

## 6. Conversión a geometría de Earth Engine

Transforma las geometrías del GeoDataFrame en una `ee.FeatureCollection`. De esta forma, cualquier notebook o script puede reconstruir la misma zona de estudio directamente desde los archivos en `data/`, sin depender de pasos manuales.

In [ ]:
# Función para convertir geometría de GeoPandas a Earth Engine
def gdf_to_ee_feature_collection(gdf):
    """Convierte un GeoDataFrame a una FeatureCollection de Earth Engine"""
    features = []
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom.geom_type == 'Polygon':
            coords = [list(geom.exterior.coords)]
        elif geom.geom_type == 'MultiPolygon':
            coords = [list(poly.exterior.coords) for poly in geom.geoms]
        
        # Crear propiedades
        properties = {
            'departamento': row['dpto_cnmbr'],
            'municipio': row['mpio_cnmbr'],
            'cod_dpto': row['dpto_ccdgo'],
            'cod_mpio': row['mpio_ccdgo']
        }
        
        # Crear feature de EE
        if geom.geom_type == 'Polygon':
            ee_geom = ee.Geometry.Polygon(coords)
        else:
            ee_geom = ee.Geometry.MultiPolygon(coords)
        
        features.append(ee.Feature(ee_geom, properties))
    
    return ee.FeatureCollection(features)

# Convertir a FeatureCollection
aoi = gdf_to_ee_feature_collection(municipios_seleccionados)
print("Área de interés convertida a Earth Engine FeatureCollection")

# Crear también una geometría única (unión de todos los municipios)
aoi_geometry = aoi.geometry()
print("Geometría unificada creada")

## 7. Definición de parámetros temporales

Define `start_date` y `end_date` para cubrir varios años agrícolas. Las series largas suavizan el efecto de semanas nubladas y permiten detectar tendencias estacionales, tal como se explica en los ejercicios de Sentinel-1 con Earth Engine [4].

In [ ]:
# Definir período de análisis
# Se recomienda usar al menos 1 año de datos para capturar ciclos agrícolas completos

# Fecha de inicio y fin (ajustar según necesidades)
fecha_inicio = '2023-01-01'  # Formato: YYYY-MM-DD
fecha_fin = '2024-12-31'

# Convertir a objetos datetime
start_date = datetime.strptime(fecha_inicio, '%Y-%m-%d')
end_date = datetime.strptime(fecha_fin, '%Y-%m-%d')

print(f"Período de análisis:")
print(f"  Inicio: {start_date.strftime('%Y-%m-%d')}")
print(f"  Fin: {end_date.strftime('%Y-%m-%d')}")
print(f"  Duración: {(end_date - start_date).days} días")

# Guardar parámetros para uso en notebooks posteriores
parametros = {
    'fecha_inicio': fecha_inicio,
    'fecha_fin': fecha_fin,
    'centroide_lat': centroide.y,
    'centroide_lon': centroide.x,
    'bounds': bounds.tolist(),
    'area_km2': area_total_km2
}

print("\nParámetros definidos y guardados")

## 8. Visualización del área de estudio

Usa geemap para dibujar los municipios seleccionados sobre un mapa base. Esta vista rápida confirma que las geometrías son correctas y ayuda a comunicar la zona de trabajo a públicos no técnicos. Puedes añadir etiquetas o marcadores si cuentas con parcelas de referencia.

In [ ]:
# Crear mapa interactivo con geemap
Map = geemap.Map(center=[centroide.y, centroide.x], zoom=9)

# Agregar la capa de municipios
Map.add_gdf(municipios_seleccionados, layer_name="Municipios de Estudio")

# Agregar el área de interés de EE
Map.addLayer(aoi, {'color': 'red'}, 'AOI - Earth Engine', opacity=0.3)

# Agregar etiquetas de municipios
for idx, row in municipios_seleccionados.iterrows():
    centroid = row.geometry.centroid
    Map.add_marker(
        location=[centroid.y, centroid.x],
        popup=f"{row['mpio_cnmbr']}, {row['dpto_cnmbr']}"
    )

# Mostrar mapa
Map

## 9. Exportar datos para notebooks posteriores

Guarda los resultados en `data/` (`municipios_seleccionados.gpkg`, parámetros temporales y configuraciones adicionales). Estos archivos funcionan como entrada directa para el preprocesamiento SAR, lo que permite reanudar el flujo sin repetir la selección geográfica.

In [ ]:
# Guardar el GeoDataFrame con los municipios seleccionados
output_path = "data/municipios_seleccionados.gpkg"
municipios_seleccionados.to_file(output_path, driver="GPKG", layer="municipios")
print(f"Municipios guardados en: {output_path}")

# Guardar parámetros en formato JSON
import json
with open('data/parametros.json', 'w') as f:
    json.dump(parametros, f, indent=2)
print(f"Parámetros guardados en: data/parametros.json")

## Resumen

Este notebook prepara la zona de estudio y los parámetros temporales que usarán los análisis posteriores. Entregamos límites municipales en WGS84, archivos listos para Earth Engine y un mapa de verificación que ayuda a alinear al equipo.

**Próximo notebook:** `02_preprocesamiento_sentinel1.ipynb` limpia y organiza la serie Sentinel-1 antes de calcular indicadores de cambio.

### Referencias bibliográficas

[1] M. J. Canty, A. A. Nielsen, H. Skriver y K. Conradsen, “Statistical Analysis of Changes in Sentinel-1 Time Series on the Google Earth Engine,” Remote Sensing, vol. 12, no. 1, p. 46, 2020, doi: 10.3390/rs12010046.

[2] J. Reiche et al., “Combining Satellite Data for Better Tropical Forest Monitoring,” Nature Climate Change, vol. 6, no. 2, pp. 120–122, 2016, doi: 10.1038/nclimate2919.

[3] M. J. Canty, *Image Analysis, Classification and Change Detection in Remote Sensing, with Algorithms for Python*, 4.ª ed., Boca Raton, FL, USA: CRC Press, 2019.

[4] Google Earth Engine Developers, “Detecting Changes in Sentinel-1 Imagery, Part 1,” Google Developers, Mountain View, CA, USA, 2024. Disponible en: https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1

[5] Ministerio de Agricultura y Desarrollo Rural, *Evaluación Agropecuaria del Departamento del Meta y Casanare*, Bogotá, Colombia, 2023.

[6] European Space Agency, *Sentinel-1 User Handbook*, ESA-EOEP-CSCOP-TN-13-0001, Noordwijk, Países Bajos, 2013.